In [31]:
#pip install -U langchain-aws

In [1]:
import boto3
import json
import os
from langchain.memory import ConversationBufferMemory
from langchain import PromptTemplate
from langchain.chains import ConversationChain
from langchain_aws import ChatBedrock
from datetime import datetime

In [2]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    region_name='us-east-1'
)

In [3]:
s3_client = boto3.client(
    's3',
    region_name='us-east-1'
)    

In [4]:
parameters = {
    "maxTokenCount":512,
    "maxTokens":1024,
    "stopSequences":["human: "],
    "temperature":0,
    "topP":0.9
    }

In [5]:
context = ConversationBufferMemory()
# context.chat_memory.add_user_message("I will act as a principal data scientist and answer data scientist questions.")
# context.chat_memory.add_ai_message("I am a principal data scientist and will answer your data scientist questions.")

In [6]:
# choosen_profile = "default"
# profile = os.environ["AWS_PROFILE"] = choosen_profile

In [7]:
prompt_template = """System: Be a humorous, precise bot.

    Current conversation:
    {history}
    Your task is to carefully read through the entire conversation history, paying close attention to the details. Identify the key information from the conversation that is relevant to answering the provided question. Concisely summarize this key relevant information in a <relevant_info> section.

    Then, based solely on the relevant information you identified, provide a precise answer to the question. Put your final answer inside <answer> tags.

    Remember, the goal is to answer the question as precisely and concisely as possible, using only the most pertinent information from the conversation history. Do not include any irrelevant details in your answer. Stay focused on directly addressing the question asked.

    Don't add any preamble statements.

    User: {input}
    Bot:"""
PROMPT = PromptTemplate(
        input_variables=["history", "input"], template=prompt_template
    )

In [8]:
claude_llm = ChatBedrock(model_id="anthropic.claude-3-sonnet-20240229-v1:0",client=bedrock_runtime
                         # ,credentials_profile_name=profile
                        )
conversation = ConversationChain(
    prompt=PROMPT,
     llm=claude_llm, verbose=False, memory=context
)

In [9]:
while True:
        input_text = input("You: ") 
        if input_text.lower() == 'exit':
            break
        chat_response = conversation.predict(input=input_text)
        print("Bot:", chat_response)  

You:  hii myself deepika


Bot: <relevant_info>The user introduced themselves as "deepika".</relevant_info>

<answer>Hello deepika!</answer>


You:  tell me some details about earth


Bot: <relevant_info>The conversation history does not contain any details about the planet Earth. The only information provided is the user's introduction, stating their name is "deepika".</relevant_info>

<answer>I do not have enough information in the conversation history to provide details about the planet Earth.</answer>


You:  fine now take a role of astronomer


...

You:  hmm nice name


Bot: <relevant_info>The user expressed appreciation for the name "Cosmo" suggested by the assistant.</relevant_info>

<answer>Thank you, I'm glad you like the name Cosmo.</answer>


You:  yeah nice to talk 


Bot: <relevant_info>The conversation so far has been a friendly exchange, with the user introducing themselves as "deepika" and their friend as "sarmistha". The assistant has taken on the role of an astronomer to provide details about the planet Earth, and has chosen the name "Cosmo" for itself after the user suggested thinking differently for a name. The user has expressed appreciation for the chosen name "Cosmo" and stated "yeah nice to talk".</relevant_info>

<answer>Thank you, it was nice to talk as well.</answer>


You:  exit


In [10]:
x=str(context.chat_memory.messages)

In [17]:
context.chat_memory.messages[1].type

'ai'

In [13]:
context

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='hii myself deepika'), AIMessage(content='<relevant_info>The user introduced themselves as "deepika".</relevant_info>\n\n<answer>Hello deepika!</answer>'), HumanMessage(content='tell me some details about earth'), AIMessage(content='<relevant_info>The conversation history does not contain any details about the planet Earth. The only information provided is the user\'s introduction, stating their name is "deepika".</relevant_info>\n\n<answer>I do not have enough information in the conversation history to provide details about the planet Earth.</answer>'), HumanMessage(content='fine now take a role of astronomer'), AIMessage(content="<relevant_info>The user has asked me to take on the role of an astronomer and provide details about the planet Earth.</relevant_info>\n\n<answer>As an astronomer, here are some key details about Earth:\n\n- Earth is the third planet from the Sun and the largest ter

In [21]:
x=str(context.chat_memory.messages)

In [23]:
x

'[HumanMessage(content=\'hii myself deepika\'), AIMessage(content=\'<relevant_info>The user introduced themselves as "deepika".</relevant_info>\\n\\n<answer>Hello deepika!</answer>\'), HumanMessage(content=\'tell me some details about earth\'), AIMessage(content=\'<relevant_info>The conversation history does not contain any details about the planet Earth. The only information provided is the user\\\'s introduction, stating their name is "deepika".</relevant_info>\\n\\n<answer>I do not have enough information in the conversation history to provide details about the planet Earth.</answer>\'), HumanMessage(content=\'fine now take a role of astronomer\'), AIMessage(content="<relevant_info>The user has asked me to take on the role of an astronomer and provide details about the planet Earth.</relevant_info>\\n\\n<answer>As an astronomer, here are some key details about Earth:\\n\\n- Earth is the third planet from the Sun and the largest terrestrial planet in our solar system.\\n- It has a ra

In [22]:
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"conversation_{timestamp}.txt"
s3_bucket_name = "rag-practice"
    
    # Write conversation data to a temporary file
with open(filename, 'w') as file:
    file.write(x)
    
# Upload the file to S3
s3_client.upload_file(filename, s3_bucket_name, filename)

# Delete the temporary file
os.remove(filename)